# Algotihms

All policies consists of Feature extraction and a FNN layer

BaseFeatureExtractor is the base class for all other Feature Extractors

stable_baselines3.common.policies

['ABC',
 'ActorCriticCnnPolicy', == stable_baselines3.ppo.CnnPolicy
 'ActorCriticPolicy', == stable_baselines3.ppo.MlpPolicy
 'Any',
 'BaseFeaturesExtractor',
 'BaseModel',
 'BasePolicy',
 'BernoulliDistribution',
 'CategoricalDistribution',
 'CombinedExtractor',
 'ContinuousCritic',
 'DiagGaussianDistribution',
 'Dict',
 'Distribution',
 'FlattenExtractor',
 'List',
 'MlpExtractor',
 'MultiCategoricalDistribution',
 'MultiInputActorCriticPolicy', == stable_baselines3.ppo.MultiInputPolicy
 'NatureCNN',
 'Optional',
 'Schedule',
 'SelfBaseModel',
 'StateDependentNoiseDistribution',
 'Tuple',
 'Type',
 'TypeVar',
 'Union',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'abstractmethod',
 'collections',
 'copy',
 'create_mlp',
 'get_action_dim',
 'get_device',
 'is_image_space',
 'is_vectorized_observation',
 'make_proba_distribution',
 'maybe_transpose',
 'nn',
 'np',
 'obs_as_tensor',
 'partial',
 'preprocess_obs',
 'spaces',
 'th',
 'warnings']

In [ ]:
dir(stable_baselines3.common.on_policy_algorithm)

['ActorCriticPolicy',
 'Any',
 'BaseAlgorithm',
 'BaseCallback',
 'Dict',
 'DictRolloutBuffer',
 'GymEnv',
 'List',
 'MaybeCallback',
 'OnPolicyAlgorithm',
 'Optional',
 'RolloutBuffer',
 'Schedule',
 'SelfOnPolicyAlgorithm',
 'Tuple',
 'Type',
 'TypeVar',
 'Union',
 'VecEnv',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'np',
 'obs_as_tensor',
 'safe_mean',
 'spaces',
 'sys',
 'th',
 'time']

Progressbar callback

In [ ]:
from stable_baselines3.common.callbacks import ProgressBarCallback

## DQN

DQN make use of different tricks to stabilize the learning with neural networks: it uses a replay buffer, a target network and gradient clipping.

In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.dqn.policies import MlpPolicy, DQNPolicy, QNetwork
from torch import nn

Below, we modify environment so we can use it for DQN !!! Delete it if it is not necessary!!!

In [ ]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious
        self.action_space = gym.spaces.Discrete(2)
        
        # All the features we have - make sure LENGTH matches your feature count
        self.observation_space = gym.spaces.Box(
            low=float('-inf'), 
            high=float('inf'), 
            shape=(LENGTH,),
            dtype=np.float32  # Specify dtype for consistency
        )
        
        self.images_per_episode = images_per_episode
        self.step_count = 0
        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
        self.expected_action = 0  # Initialize this
        
    def step(self, action):
        # Calculate reward
        reward = float(action == self.expected_action)  # Convert to float
        current_label = self.expected_action
        
        # Get next observation
        obs = self._next_obs()
        
        # Update step count
        self.step_count += 1
        done = self.step_count >= self.images_per_episode
        
        # Standard gym format: obs, reward, done, info
        info = {
            'label': current_label,
            'TimeLimit.truncated': done and self.step_count >= self.images_per_episode
        }
        
        return obs, reward, done, info, {}
    
    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx])
            obs = self.x.iloc[next_obs_idx].values  # Convert to numpy array
        else:
            if self.dataset_idx >= len(self.x):
                # Reset to beginning if we've reached the end
                self.dataset_idx = 0
            
            obs = self.x.iloc[self.dataset_idx].values  # Convert to numpy array
            self.expected_action = int(self.y.iloc[self.dataset_idx])
            self.dataset_idx += 1
            
        # Ensure the observation is the right shape and type
        return obs.astype(np.float32)
    
    def reset(self, seed: Optional[int] = None, options: Optional[dict] = None):
        super().reset(seed=seed)
        self.step_count = 0
        self.dataset_idx = 0 if not self.random else self.dataset_idx
        
        obs = self._next_obs()
        info = {}  # Return empty dict for info
        
        return obs, info

In [ ]:
model = DQN(
    policy='MlpPolicy',#CustomPolicy
    env=env,
    gamma=1, #Discount factor
    learning_rate=0.00025,  #Progress decreases from 1 to 0 -> lr decreasesb from 0.0021 to 0
    # n_steps =512, #The number of steps to run for each environment per update (i.e. rollout buffer size is n_steps * n_envs where n_envs is number of environment copies running in parallel) NOTE: n_steps * n_envs must be greater than 1 (because of the advantage normalization) See
    max_grad_norm=0.8, #The maximum value for the gradient clipping
    verbose=0,
    tensorboard_log="log_25", #tensorboard log location
    # gae_lambda=0.8, #Factor for trade-off of bias vs variance for Generalized Advantage Estimato
    batch_size=32, #minibatch size
    exploration_fraction=0.1,
    exploration_final_eps=0.1,
    train_freq=4,
    learning_starts=5000,
    # buffer_size=1000000,
    buffer_size = 1000
    # target_network_update_freq=10000,
    #normalize_advantage
)

the default policy depends on algorithm and observation space

In [ ]:
model.policy

In [ ]:
eval_callback = AccF1Callback(df_train, df_test, eval_freq=1000)

In [ ]:
model.learn(
    total_timesteps=5000000, #total number of samples(environment steps) to train on
    callback=eval_callback,progress_bar=True,
    # log_interval = int(1.0e4)
    )

In [ ]:
clear_progress_bar()

In [ ]:
df_train.random(4).apply(predict_random_sample, axis=1)

Feature Extractor redefined with net_arch

In [ ]:
class CustomFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space):
        super(CustomFeatureExtractor, self).__init__(observation_space, features_dim=32)
        self.network = nn.Sequential(
            nn.Linear(observation_space.shape[0], 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
        )

    def forward(self, observations: torch.Tensor):
        # raise ValueError("forward method was called")
        return self.network(observations)


class CustomMlpPolicy(DQNPolicy):
    def __init__(self, *args, **kwargs):
        features_extractor_class = CustomFeatureExtractor
        net_arch = [64,5]
        super(CustomMlpPolicy, self).__init__(*args, **kwargs,net_arch = net_arch, features_extractor_class=features_extractor_class)

In [ ]:
model.policy

Batch Normaliztion in 1d with net_arch

In [ ]:
class CustomFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space):
        super(CustomFeatureExtractor, self).__init__(observation_space, features_dim=32)
        self.network = nn.Sequential(
            nn.Linear(observation_space.shape[0], 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

    def forward(self, observations: torch.Tensor):
        # raise ValueError("forward method was called")
        return self.network(observations)


class CustomMlpPolicy(DQNPolicy):
    def __init__(self, *args, **kwargs):
        features_extractor_class = CustomFeatureExtractor
        net_arch = [64,5]
        super(CustomMlpPolicy, self).__init__(*args, **kwargs,net_arch = net_arch, features_extractor_class=features_extractor_class)

In [ ]:
model.policy

TabTransform in Feature Extractor with net_arch

In [ ]:
class CustomFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space):
        super(CustomFeatureExtractor, self).__init__(observation_space, features_dim=32)
        self.embedding = nn.Linear(observation_space.shape[0], 32)
        encoder_layer = nn.TransformerEncoderLayer(d_model=32, nhead=4, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=4)



    def forward(self, observations: torch.Tensor):
        x = self.embedding(observations)
        x = x.unsqueeze(1) 
        x = self.transformer(x)
        x = torch.mean(x, dim=1)  
        return x

class CustomMlpPolicy(DQNPolicy):
    def __init__(self, *args, **kwargs):
        features_extractor_class = CustomFeatureExtractor
        net_arch = [64,5]
        super(CustomMlpPolicy, self).__init__(*args, **kwargs,net_arch = net_arch, features_extractor_class=features_extractor_class)

In [ ]:
model.policy

Redefine Policy Network in Complete

In [ ]:
class CustomQNetwork(QNetwork):
  def __init__(
        self,
        observation_space: spaces.Space,
        action_space: spaces.Discrete,
        features_extractor: BaseFeaturesExtractor,
        features_dim: int,
        net_arch: Optional[list[int]] = None,
        activation_fn: type[nn.Module] = nn.ReLU,
        normalize_images: bool = True,
    ) -> None:
        super(CustomQNetwork,self).__init__(
            observation_space,
            action_space,
            features_dim=features_dim,
            features_extractor=features_extractor,
            normalize_images=normalize_images,
        )



        # self.net_arch = net_arch
        # self.activation_fn = activation_fn
        # self.features_dim = features_dim
        # action_dim = int(self.action_space.n)  # number of actions
        # q_net = create_mlp(self.features_dim, action_dim, self.net_arch, self.activation_fn)
        self.q_net = nn.Sequential(
            nn.Linear(32, 16), nn.ReLU(),
            nn.Linear(16 , 8), nn.ReLU(),
            nn.Linear(8,2), nn.ReLU()
        )

In [ ]:
class CustomFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space):
        super(CustomFeatureExtractor, self).__init__(observation_space, features_dim=32)
        self.network = nn.Sequential(
            nn.Linear(observation_space.shape[0], 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
        )

    def forward(self, observations: torch.Tensor):
        # raise ValueError("forward method was called")
        return self.network(observations)


class CustomMlpPolicy(DQNPolicy):
    def __init__(self, *args, **kwargs):
        features_extractor_class = CustomFeatureExtractor
        super(CustomMlpPolicy, self).__init__(*args, **kwargs, features_extractor_class=features_extractor_class)
    def make_q_net(self):
       net_args = self._update_features_extractor(self.net_args)
       return CustomQNetwork(**net_args).to(self.device)

In [ ]:
model.policy

In [ ]:
clear_progress_bar()
model.learn(
    total_timesteps=100000, #total number of samples(environment steps) to train on
    callback=eval_callback,progress_bar=True,
    # log_interval = int(1.0e4)
)

In [ ]:
predicted = df_test.apply(predict_row, axis=1)
accuracy = accuracy_score(list(df_test["labels"]), predicted.to_list())
f1 = f1_score(list(df_test["labels"]), predicted.to_list())
print("Validation --- Accuracy: ", accuracy)
print("Validation --- F1-Score: ", f1)

print("*-" * 50)

## PPO

In [ ]:
from stable_baselines3 import PPO

It combines the idea of TRPO (using a trust region to ensure after each upadte, the new policy doesn't go far from the old one) and A2C (multiple agents)

##### Default Policy

In [ ]:
model = PPO(
    policy='MlpPolicy',#CustomPolicy
    env=env,
    gamma=0.9, #Discount factor
    learning_rate=lambda progress: progress * 0.0021,  #Progress decreases from 1 to 0 -> lr decreasesb from 0.0021 to 0
    n_steps=512, #The number of steps to run for each environment per update (i.e. rollout buffer size is n_steps * n_envs where n_envs is number of environment copies running in parallel) NOTE: n_steps * n_envs must be greater than 1 (because of the advantage normalization) See
    ent_coef=1e-05, #Entropy coefficient for the loss calculation
    vf_coef=0.6, #Value function coefficient for the loss calculation
    max_grad_norm=0.8, #The maximum value for the gradient clipping
    clip_range=0.2, #Clipping parameter, it can be a function of the current progress remaining (from 1 to 0)
    verbose=0,
    tensorboard_log="log_25", #tensorboard log location
    gae_lambda=0.8, #Factor for trade-off of bias vs variance for Generalized Advantage Estimato
    batch_size=16, #minibatch size
    n_epochs=55 #Number of epoch when optimizing the surrogate loss
    #normalize_advantage
)

In [ ]:
model.learn(
    total_timesteps=5000000, #total number of samples(environment steps) to train on
    callback=eval_callback,progress_bar=True)

Subprocess run

PPO is meant to be run primarily on the CPU, especially when you are not using a CNN. To improve CPU utilization, try turning off the GPU and using **SubprocVecEnv** instead of the default **DummyVecEnv**:

In [ ]:
stable_baselines3.common.env_util import make_vec_env

env = make_vec_env("CartPole-v1", n_envs=8, vec_env_cls=SubprocVecEnv)
model = PPO("MlpPolicy", env, device="cpu")
model.learn(total_timesteps=25_000)

Save/Load trained model

In [ ]:
model.save("ppo_cartpole")
model = PPO.load("ppo_cartpole")

#### Custom Policy Network 1

In [ ]:
import torch as th

In [ ]:
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=[122, 64, 32],
                     # features_extractor_class="MlpExtractor",
                     # features_extractor_kwargs=dict(features_dim=122)
                    )

In [ ]:
model = PPO(
    policy='MlpPolicy',#CustomPolicy
    env=env,
    gamma=0.9, #Discount factor
    learning_rate=lambda progress: progress * 0.0021,  #Progress decreases from 1 to 0 -> lr decreasesb from 0.0021 to 0
    n_steps=512, #The number of steps to run for each environment per update (i.e. rollout buffer size is n_steps * n_envs where n_envs is number of environment copies running in parallel) NOTE: n_steps * n_envs must be greater than 1 (because of the advantage normalization) See
    ent_coef=1e-05, #Entropy coefficient for the loss calculation
    vf_coef=0.6, #Value function coefficient for the loss calculation
    max_grad_norm=0.8, #The maximum value for the gradient clipping
    clip_range=0.2, #Clipping parameter, it can be a function of the current progress remaining (from 1 to 0)
    verbose=0,
    tensorboard_log="log_25", #tensorboard log location
    gae_lambda=0.8, #Factor for trade-off of bias vs variance for Generalized Advantage Estimato
    batch_size=16, #minibatch size
    n_epochs=55, #Number of epoch when optimizing the surrogate loss
    policy_kwargs=policy_kwargs
    #normalize_advantage
)

## A2C

A synchronous, deterministic variant of Asynchronous Advantage Actor Critic (A3C). It uses multiple workers to avoid the use of a replay buffer.

In [ ]:
from stable_baselines3 import A2C

In [ ]:
eval_callback = AccF1Callback(df_train, df_test, eval_freq=100000)

### Simple Environment

In [ ]:
model = A2C("MlpPolicy",
            env=env, 
            gamma=0.9, #Discount factor
            learning_rate=lambda progress: progress * 0.0021,  #Progress decreases from 1 to 0 -> lr decreasesb from 0.0021 to 0
            n_steps=512, #The number of steps to run for each environment per update (i.e. rollout buffer size is n_steps * n_envs where n_envs is number of environment copies running in parallel) NOTE: n_steps * n_envs must be greater than 1 (because of the advantage normalization) See
            ent_coef=1e-05, #Entropy coefficient for the loss calculation
            vf_coef=0.6, #Value function coefficient for the loss calculation
            max_grad_norm=0.8, #The maximum value for the gradient clipping
            verbose=0,
            tensorboard_log="log_25", #tensorboard log location
            gae_lambda=0.8, #Factor for trade-off of bias vs variance for Generalized Advantage Estimato
            #normalize_advantage
            device="cpu")

In [ ]:
clear_progress_bar()
model.learn(
    total_timesteps=100000, #total number of samples(environment steps) to train on
    callback=eval_callback,progress_bar=True)

### Vectorized Environment (Multi-Agent)

In [ ]:
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from typing import Callable
from stable_baselines3.common.utils import set_random_seed

In [ ]:
def make_env(rank: int, seed: int = 0) -> Callable:
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environment you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    :return: (Callable)
    """

    def _init() -> gym.Env:
        # ev = gym.make(env)
        ev = IdsEnv()
        ev.reset(seed=seed + rank)
        return ev

    set_random_seed(seed)
    return _init

In [ ]:
num_cpu = 4

#### DummyVec

In [ ]:
envv = DummyVecEnv([make_env(i) for i in range(num_cpu)])

#### SubprocVec

In [ ]:
# Environment
import gymnasium as gym
from gymnasium.utils.env_checker import check_env
import random
from typing import Optional, Callable
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1,dataset=(x_train, y_train), random=True):
        self.action_space = gym.spaces.Discrete(2) ## normal or malicious
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(len(x_train.columns),))
        self.step_count = 0
        self.images_per_episode = images_per_episode
        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0

    def step(self, action):
        done = False
        reward = int(action == self.expected_action)
        current_label = self.expected_action
        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {} ,{'label': current_label}

        ###### for algorithms with replay buffer as bellow
        # info = {
        #     'label': current_label,
        #     'TimeLimit.truncated': done and self.step_count >= self.images_per_episode
        # }
        # return obs, reward, done, info, {}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()
        return th.as_tensor(obs,device="cpu")

    def reset(self,seed: Optional[int] = None, options: Optional[dict] = None):
        super().reset(seed=seed)


        self.step_count = 0
        obs = self._next_obs()
        return obs,{0:"info"}
env = IdsEnv()
# check_env(env)
env.reset()
# If you want to train the algorithm on multiple environment in parallel through the stable-baselines lib (vectorized environments)
# from stable_baselines3.common.vec_env import DummyVecEnv
# n_envs = 16  # hyperparameter
# env = DummyVecEnv([lambda: IdsEnv()] * n_envs)

In [ ]:
envv = SubprocVecEnv([make_env(i) for i in range(num_cpu)])

In [ ]:
model = A2C("MlpPolicy",
            env=env, 
            gamma=0.9, #Discount factor
            learning_rate=lambda progress: progress * 0.0021,  #Progress decreases from 1 to 0 -> lr decreasesb from 0.0021 to 0
            n_steps=512, #The number of steps to run for each environment per update (i.e. rollout buffer size is n_steps * n_envs where n_envs is number of environment copies running in parallel) NOTE: n_steps * n_envs must be greater than 1 (because of the advantage normalization) See
            ent_coef=1e-05, #Entropy coefficient for the loss calculation
            vf_coef=0.6, #Value function coefficient for the loss calculation
            max_grad_norm=0.8, #The maximum value for the gradient clipping
            verbose=0,
            tensorboard_log="log_25", #tensorboard log location
            gae_lambda=0.8, #Factor for trade-off of bias vs variance for Generalized Advantage Estimato
            #normalize_advantage
            device="cpu")

In [ ]:
clear_progress_bar()
model.learn(
    total_timesteps=100000, #total number of samples(environment steps) to train on
    callback=eval_callback,progress_bar=True)

In [ ]:
envv.processes

In [ ]:
envv.num_envs

### Evaluate Model

In [ ]:
# We create a separate environment for evaluation
eval_env = IdsEnv()

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward:.2f}")

In [ ]:
predicted = df_test.apply(predict_row, axis=1)
accuracy = accuracy_score(list(df_test["labels"]), predicted.to_list())
f1 = f1_score(list(df_test["labels"]), predicted.to_list())
print("Validation --- Accuracy: ", accuracy)
print("Validation --- F1-Score: ", f1)
            
print("*-" * 50)

# Util Functions

#### Predict random sample

In [ ]:
def predict_random_sample(s):
    print(s.labels.to_numpy(dtype=np.int64))
    y = torch.tensor(s["labels"].to_numpy(dtype=np.float32), dtype=torch.float)
    f = torch.tensor(s.drop(columns=["labels"]).to_numpy(dtype=np.float32), dtype=torch.float)
    predicted = model.predict(f,deteministic=True)[0]
    print(y, predicted)
    print("Acc: ",accuracy_score(y, predicted))

#### Clear progressbars

In [ ]:
import gc
import tqdm
import rich

def clear_progress_bar():
    # On cherche tous les objets dont le nom du type contient tqdm
    tqdm_objects = [obj for obj in gc.get_objects() if 'tqdm' in type(obj).__name__]
    
    # On affiche leur type
    for tqdm_object in tqdm_objects:
        print(type(tqdm_object).__name__)
    
    # On ferme ceux qu'on veut
    for tqdm_object in tqdm_objects:
        if 'tqdm_rich' in type(tqdm_object).__name__:
            try:
                tqdm_object.close()
            except:
                continue

#### Gym Callbacks

Train/Test Evalution on Frequency Interval

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
def predict_row(row):
            features = torch.tensor(row.drop("labels").to_numpy(dtype=np.float32), dtype=torch.float32)
            f =  model.predict(features, deterministic=True)[0]
            return f
    
class AccF1Callback(BaseCallback):
    def __init__(self,train, val, eval_freq,verbose:int=0 ):
        super().__init__(verbose)
        self.train_data = train
        self.val_data = val
        self.eval_freq = eval_freq
        # The RL model
        # self.model = None  # type: BaseAlgorithm
        # An alias for self.model.get_env(), the environment used for training
        # self.training_env # type: VecEnv
        # Number of time the callback was called
        # self.n_calls = 0  # type: int
        # num_timesteps = n_envs * number of times env.step() was called
        # self.num_timesteps = 0  # type: int
    
    # def _on_training_start(self) -> None:
    #     """
    #     This method is called before the first rollout starts.
    #     """
    #     pass

    
    # def _on_training_end(self) -> None:
    #     """
    #     This event is triggered before exiting the `learn()` method.
    #     """
    #     pass
    

    # def _on_rollout_start(self) -> None:
    #     """
    #     A rollout is the collection of environment interaction
    #     using the current policy.
    #     This event is triggered before collecting new samples.
    #     """
    #     pass

    # def _on_rollout_end(self) -> None:
    #     """
    #     This event is triggered before updating the policy.
    #     """
    #     pass

    
        
    def _on_step(self):
        """
        This method will be called by the model after each call to `env.step()`.

        For child callback (of an `EventCallback`), this will be called
        when the event is triggered.

        :return: If the callback returns False, training is aborted early.
        """


        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()

            predicted = self.train_data.apply(predict_row, axis=1)
            accuracy = accuracy_score(list(self.train_data["labels"]), predicted.to_list())
            f1 = f1_score(list(self.train_data["labels"]), predicted.to_list())

            print("*-" * 50)
            
            print("total current timesteps: ", self.num_timesteps, '\n')
            print("Training --- Accuracy: ", accuracy)
            print("Training --- F1-Score: ", f1, '\n')

            predicted = self.val_data.apply(predict_row, axis=1)
            accuracy = accuracy_score(list(self.val_data["labels"]), predicted.to_list())
            f1 = f1_score(list(self.val_data["labels"]), predicted.to_list())
            print("Validation --- Accuracy: ", accuracy)
            print("Validation --- F1-Score: ", f1)
            
            print("*-" * 50)

        return True